In [2]:
# %matplotlib inline
%matplotlib widget

import sys,os
sys.path.append(os.getcwd())

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

import cvxpy as cp
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from qpth.qp import QPFunction, QPSolvers

import warnings
warnings.filterwarnings('ignore')

%run BallPaddleSystem.py
%run ParametrizedQP.py
%run utils.py

In [3]:
system = BallPaddleSystem(dt=.05,u_max=2.)
ball_x0_min = np.array([0.25, -1.5])
ball_x0_max = np.array([1.75, 2.])
paddle_x0 = np.array([0.,0.])
ball_xg = np.array([1.,0.])
N = 20

# system = BallPaddleSystem(dt=.1,u_max=10.)
# ball_x0_min = np.array([0.05, -10.])
# ball_x0_max = np.array([2., 10.])
# paddle_x0 = np.array([0.,0.])
# ball_xg = np.array([1.,0.])
# N = 5

### Generate data of the cost-to-go

In [4]:
dim_pos = 10
dim_vel = 10
ball_x0_pos = np.linspace(ball_x0_min[0], ball_x0_max[0], dim_pos)
ball_x0_vel = np.linspace(ball_x0_min[1], ball_x0_max[1], dim_vel)
BALL_POS, BALL_VEL = np.meshgrid(ball_x0_pos, ball_x0_vel)

In [5]:
BALL_VAL = np.zeros(BALL_POS.shape)
for i in range(BALL_POS.shape[0]):
    for j in range(BALL_POS.shape[1]):
        (prob,objective,constraints,var) = system.get_trajectory_miqp(paddle_x0,[BALL_POS[i,j],BALL_VEL[i,j]],ball_xg,N)
        prob.solve(solver=cp.CPLEX)
        val = objective.value
        if val:
            BALL_VAL[i,j] = val
        else:
            BALL_VAL[i,j] = None
            
        update_progress((i*BALL_POS.shape[1]+j+1)/(BALL_POS.shape[0]*BALL_POS.shape[1]))

Progress: [########################################] 100.0%


In [ ]:
# save training data
np.savez('ball_paddle_values', BALL_VAL=BALL_VAL)

In [ ]:
# OR load training data
data = np.load('ball_paddle_values.npz')
BALL_VAL = data['BALL_VAL']

In [7]:
# clean the data
np.nan_to_num(BALL_VAL,copy=False,nan=np.nanmax(BALL_VAL));

In [8]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(BALL_POS, BALL_VEL, BALL_VAL, rstride=1, cstride=1,
                cmap='plasma', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
xy_data = np.vstack((np.reshape(BALL_POS,-1),np.reshape(BALL_VEL,-1))).T
z_label = np.expand_dims(np.reshape(BALL_VAL,-1),axis=1)

# shuffle it
num_data = xy_data.shape[0]
indx = np.arange(num_data)
np.random.shuffle(indx)

xy_data = xy_data[indx,:]
z_label = z_label[indx,:]

### Train a neural network to approximate it

In [10]:
learning_rate = 1e-3
batch_size = 50

nn_width = 24
model = torch.nn.Sequential(
    torch.nn.Linear(2, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, 1)
)
model.double()

loss_fn = torch.nn.MSELoss(reduction='sum')
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

writer = SummaryWriter()
n_iter = 0

In [ ]:
# optional: move to GPU
model.cuda()

In [ ]:
device = next(model.parameters()).device
xy_data_tensor = torch.from_numpy(xy_data).to(device)
z_label_tensor = torch.from_numpy(z_label).to(device)

num_epoch = 10000

for epoch in range(num_epoch):
    batch_start = 0
    while batch_start < num_data:
        batch_end = min(num_data-1,batch_start+batch_size)
        x = xy_data_tensor[batch_start:batch_end,:]
        z = z_label_tensor[batch_start:batch_end,:]
        z_pred = model(x)
        loss = loss_fn(z_pred, z) / batch_size
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_start += batch_size
        n_iter += 1
    if epoch % 10 == 0:
        print("loss: %f" % loss.item())
#         writer.add_scalar('Loss/train', loss.item(), n_iter)

In [ ]:
writer.close()

#### Find where the neural network over-approximates the most (becomes inadmissible)

In [14]:
(prob,objective,constraints,var) = system.get_adversarial_miqp(model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
prob.solve(solver=cp.CPLEX)

worst_input = np.array(var['zb'].value[:,0])

print(objective.value)

-26.18005452392986


In [18]:
# check if the relaxed problem is far off
(prob,objective,constraints,var) = system.get_adversarial_qp(model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
prob.solve(verbose=True,solver=cp.CPLEX)

print(objective.value)

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
QP Presolve eliminated 376 rows and 20 columns.
Aggregator did 81 substitutions.
Reduced QP has 284 rows, 166 columns, and 2956 nonzeros.
Reduced QP objective Q matrix has 19 nonzeros.
Presolve time = 0.00 sec. (0.54 ticks)
Parallel mode: using up to 32 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 9050
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.44 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 32
  Rows in Factor            = 284
  Integer space required    = 554
  Total non-zeros in factor = 10056
  Total FP ops to factor    = 541072
 Itn      Primal Obj        Dual Obj  Prim Inf Upper Inf  Dual Inf          
   0   4.8962156e+01  -3.2539202e+05  4.15e+03  2.23e+03  2.38e+05
   1  -1.7052292e+01  -3.2448917e+05  3.44e+02  1.85e+02  1.98e+04
   2  -1.7845845e+01  -1.0035874e+05  2.02e+01  1.08e+01  1.16e+03
   3

In [17]:
# and diff solver accurate
prob = system.get_adversarial_qp_standard(model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
qp_fun = QPFunction(verbose=False,solver=QPSolvers.CVXPY,check_Q_spd=True)
x_adv = qp_fun(prob.Q + 1e-6*torch.eye(prob.num_vars).type(torch.DoubleTensor).to(device), prob.q, prob.G, prob.h, prob.A, prob.b)
r = prob.eval_obj(x_adv)

print(r.item())

-90.75892779186107


### Retrain the network but with admissibility regularization and check again

In [ ]:
learning_rate = 1e-3
batch_size = 50

nn_width = 24
admissible_model = torch.nn.Sequential(
    torch.nn.Linear(2, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, 1)
)
admissible_model.double()

loss_fn = torch.nn.MSELoss(reduction='sum')
# optimizer = torch.optim.SGD(admissible_model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(admissible_model.parameters(), lr=learning_rate)

writer = SummaryWriter()
n_iter = 0

In [ ]:
learning_rate = 1e-4
optimizer = torch.optim.SGD(admissible_model.parameters(), lr=learning_rate)

In [ ]:
# optional: move to GPU
admissible_model.cuda()

In [ ]:
device = next(admissible_model.parameters()).device
xy_data_tensor = torch.from_numpy(xy_data).to(device)
z_label_tensor = torch.from_numpy(z_label).to(device)

num_epoch = 100

qp_fun = QPFunction(verbose=False,solver=QPSolvers.CVXPY,check_Q_spd=True)

for epoch in range(num_epoch):
    batch_start = 0
    
    while batch_start < num_data:
        batch_end = min(num_data-1,batch_start+batch_size)
        x = xy_data_tensor[batch_start:batch_end,:]
        z = z_label_tensor[batch_start:batch_end,:]
        z_pred = admissible_model(x)
        
        prob = system.get_adversarial_qp_standard(admissible_model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
        
        QI = 1e-6*torch.eye(prob.num_vars).to(device).type(prob.dtype)
        x_adv = qp_fun(prob.Q + QI, prob.q, prob.G, prob.h, prob.A, prob.b)
        r = prob.eval_obj(x_adv) * 10.
    
        reg_loss = F.relu(-r)
        fit_loss = loss_fn(z_pred, z) / batch_size 
    
        loss = fit_loss + reg_loss
    
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        
        batch_start += batch_size
    
    if epoch % 10 == 0:
#         print("reg: %f" % reg_loss.item())
#         print("fit: %f" % fit_loss.item())
#         print("loss: %f" % loss.item())
        writer.add_scalar('Admissible/train', loss.item(), n_iter)

### Trying with adversarial activations

In [ ]:
# trying the activation-based approach

learning_rate = 1e-3
batch_size = 50

# nn_width = 12
# admissible_model = torch.nn.Sequential(
#     torch.nn.Linear(2, nn_width),
#     torch.nn.ReLU(),
#     torch.nn.Linear(nn_width, nn_width),
#     torch.nn.ReLU(),
#     torch.nn.Linear(nn_width, 1)
# )
# nn_width = 48
# admissible_model = torch.nn.Sequential(
#     torch.nn.Linear(2, nn_width),
#     torch.nn.ReLU(),
#     torch.nn.Linear(nn_width, nn_width),
#     torch.nn.ReLU(),
#     torch.nn.Linear(nn_width, nn_width),
#     torch.nn.ReLU(),
#     torch.nn.Linear(nn_width, 1)
# )
# admissible_model.double()
nn_width = 24
admissible_model = torch.nn.Sequential(
    torch.nn.Linear(2, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, nn_width),
    torch.nn.ReLU(),
    torch.nn.Linear(nn_width, 1)
)

loss_fn = torch.nn.MSELoss(reduction='sum')
# optimizer = torch.optim.SGD(admissible_model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(admissible_model.parameters(), lr=learning_rate)

In [ ]:
learning_rate = 1e-4
optimizer = torch.optim.SGD(admissible_model.parameters(), lr=learning_rate)

In [ ]:
# optional: move to GPU
admissible_model.cuda()

In [ ]:
# find an adverserial example
with torch.no_grad():
    (prob,objective,constraints,var) = system.get_adversarial_miqp(admissible_model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
    prob.solve(solver=cp.CPLEX)
    bi = np.array(var['bi'].value)
    v = [np.array(v.value) for v in var['v']]
    #print("adv: %f" % objective.value)

In [ ]:
device = next(admissible_model.parameters()).device
xy_data_tensor = torch.from_numpy(xy_data).to(device)
z_label_tensor = torch.from_numpy(z_label).to(device)

num_epoch = 100

qp_fun = QPFunction(verbose=False,solver=QPSolvers.CVXPY,check_Q_spd=True)

for epoch in range(num_epoch):
    batch_start = 0
    
    while batch_start < num_data:
        batch_end = min(num_data-1,batch_start+batch_size)
        x = xy_data_tensor[batch_start:batch_end,:]
        z = z_label_tensor[batch_start:batch_end,:]
        z_pred = admissible_model(x)
        fit_loss = loss_fn(z_pred, z) / batch_size
        
#         # find an adverserial example
#         with torch.no_grad():
#             (prob,objective,constraints,var) = system.get_adversarial_miqp(admissible_model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
#             prob.solve(solver=cp.CPLEX)
#             bi = np.array(var['bi'].value)
#             v = [np.array(v.value) for v in var['v']]
#             #print("adv: %f" % objective.value)
            
        prob = system.get_adversarial_qp_standard(admissible_model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N,bi=bi,v=v)    
        QI = 1e-6*torch.eye(prob.num_vars).to(device).type(prob.dtype)
        x_adv = qp_fun(prob.Q + QI, prob.q, prob.G, prob.h, prob.A, prob.b)
        r = prob.eval_obj(x_adv) * 10.
        #print("relaxed adv: %f" % r.item())
        reg_loss = F.relu(-r)
                
        loss = fit_loss + reg_loss
    
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        
        batch_start += batch_size
    
    if epoch % 10 == 0:
        print("reg: %f" % reg_loss.item())
        print("fit: %f" % fit_loss.item())
        print("loss: %f" % loss.item())

In [ ]:
(prob,objective,constraints,var) = system.get_adversarial_miqp(admissible_model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
prob.solve(solver=cp.CPLEX)

worst_input_admiss = np.array(var['zb'].value[:,0])

print(objective.value)

In [ ]:
(prob,objective,constraints,var) = system.get_adversarial_qp(admissible_model,paddle_x0,ball_x0_min,ball_x0_max,ball_xg,N)
prob.solve()

print(objective.value)

### Plotting the samples and the approximated cost-to-go

In [ ]:
print("=== NOT ADMISSIBLE ===")

print("Neural network")
print(worst_input)
with torch.no_grad():
    worst_value = model(torch.from_numpy(worst_input).to(device)).cpu().numpy()
print(worst_value)

print("Optimization problem")
print(worst_input)
(prob,objective,constraints,var) = system.get_trajectory_miqp(paddle_x0,worst_input,ball_xg,N)
prob.solve(solver=cp.CPLEX)
worst_opt_val = objective.value
print(worst_opt_val)

print("Optimization of closest on grid")
worst_input_grid = [find_nearest(ball_x0_pos,worst_input[0]),find_nearest(ball_x0_vel,worst_input[1])]
print(worst_input_grid)
(prob,objective,constraints,var) = system.get_trajectory_miqp(paddle_x0,worst_input_grid,ball_xg,N)
prob.solve(solver=cp.CPLEX)
worst_opt_val_grid = objective.value
print(worst_opt_val_grid)

print("Neural network of closest on grid")
print(worst_input_grid)
with torch.no_grad():
    worst_value_grid = model(torch.from_numpy(np.array(worst_input_grid)).to(device)).cpu().numpy()
print(worst_value_grid)

print("\n=== ADMISSIBLE ===")

print("Admissible neural network (same input as nonadmissible)")
print(worst_input)
with torch.no_grad():
    worst_value_same = admissible_model(torch.from_numpy(worst_input).to(device)).cpu().numpy()
print(worst_value_same)

print("Admissible neural network (worst for this network)")
print(worst_input_admiss)
with torch.no_grad():
    worst_value_admiss = admissible_model(torch.from_numpy(worst_input_admiss).to(device)).cpu().numpy()
print(worst_value_admiss)

print("Optimization problem")
print(worst_input_admiss)
(prob,objective,constraints,var) = system.get_trajectory_miqp(paddle_x0,worst_input_admiss,ball_xg,N)
prob.solve(solver=cp.CPLEX)
worst_opt_val_admiss = objective.value
print(worst_opt_val_admiss)

print("Optimization of closest on grid")
worst_input_grid_admiss = [find_nearest(ball_x0_pos,worst_input_admiss[0]),find_nearest(ball_x0_vel,worst_input_admiss[1])]
print(worst_input_grid_admiss)
(prob,objective,constraints,var) = system.get_trajectory_miqp(paddle_x0,worst_input_grid_admiss,ball_xg,N)
prob.solve(solver=cp.CPLEX)
worst_opt_val_admiss_grid = objective.value
print(worst_opt_val_admiss_grid)

print("Neural network of closest on grid")
print(worst_input_grid_admiss)
with torch.no_grad():
    worst_value_admiss_grid = admissible_model(torch.from_numpy(np.array(worst_input_grid_admiss)).to(device)).cpu().numpy()
print(worst_value_admiss_grid)

In [12]:
with torch.no_grad():
    z_pred = model(torch.from_numpy(xy_data).to(device)).cpu().numpy()
#     z_pred_admiss = admissible_model(torch.from_numpy(xy_data).to(device)).cpu().numpy()
    
# unshuffle
z_pred = z_pred[[np.argwhere(indx == i)[0,0] for i in np.arange(num_data)]]
Z_pred = np.reshape(z_pred,BALL_POS.shape)

# z_pred_admiss = z_pred_admiss[[np.argwhere(indx == i)[0,0] for i in np.arange(num_data)]]
# Z_pred_admiss = np.reshape(z_pred_admiss,BALL_POS.shape)

In [13]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(BALL_POS, BALL_VEL, BALL_VAL, rstride=1, cstride=1,
                cmap='plasma', edgecolor='none')
ax.plot_surface(BALL_POS, BALL_VEL, Z_pred, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
# ax.plot_surface(BALL_POS, BALL_VEL, Z_pred_admiss, rstride=1, cstride=1,
#                 cmap='Greys', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(BALL_POS, BALL_VEL, BALL_VAL - Z_pred_admiss, rstride=1, cstride=1,
                cmap='plasma', edgecolor='none')